# Precificação de Veículos Seminovos com Machine Learning

Queremos construir um modelo para **estimar o preço de venda (`sale_price`) de carros usados na Índia**, usando atributos como ano de fabricação, quilometragem, marca, modelo, etc.

Para evitar modelos com pouquíssimos exemplos, vamos **restringir o dataset** apenas aos modelos de carro (`model`) que aparecem **pelo menos 100 vezes**. Assim, o modelo aprende melhor e nossas conclusões valem apenas para esses modelos mais comuns.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração básica de gráficos
plt.style.use("default")
sns.set(rc={"figure.figsize": (10, 5)})

# 1) Carregar dataset original
df_raw = pd.read_csv("Used_Car_Price_Prediction.csv")

print("Shape original:", df_raw.shape)

# 2) Manter apenas modelos com pelo menos 100 ocorrências
model_counts = df_raw["model"].value_counts()
models_to_keep = model_counts[model_counts >= 100].index

df = df_raw[df_raw["model"].isin(models_to_keep)].copy()

print("Shape após recorte por modelo:", df.shape)
print(f"Proporção de linhas mantidas: {len(df) / len(df_raw):.2%}")

Shape original: (7400, 29)
Shape após recorte por modelo: (5316, 29)
Proporção de linhas mantidas: 71.84%


In [5]:
print("Primeiras 5 linhas do dataset filtrado:")
display(df.head())

print("\nInformações gerais (types, não nulos):")
df.info()

Primeiras 5 linhas do dataset filtrado:


,car_name,yr_mfr,fuel_type,kms_run,sale_price,city,times_viewed,body_type,transmission,variant,...,total_owners,broker_quote,original_price,car_rating,ad_created_on,fitness_certificate,emi_starts_from,booking_down_pymnt,reserved,warranty_avail
0,maruti swift,2015,petrol,8063,386399,noida,18715,hatchback,manual,lxi opt,...,2,397677,404177.0,great,2021-04-04T07:09:18.583,True,8975,57960,False,False
1,maruti alto 800,2016,petrol,23104,265499,noida,2676,hatchback,manual,lxi,...,1,272935,354313.0,great,2021-03-22T14:07:32.833,True,6167,39825,False,False
2,hyundai grand i10,2017,petrol,23402,477699,noida,609,hatchback,manual,sports 1.2 vtvt,...,1,469605,NaN,great,2021-03-20T05:36:31.311,True,11096,71655,False,False
3,maruti swift,2013,diesel,39124,307999,noida,6511,hatchback,manual,vdi,...,1,294262,374326.0,great,2021-01-21T12:59:19.299,True,7154,46200,False,False
4,hyundai grand i10,2015,petrol,22116,361499,noida,3225,hatchback,manual,magna 1.2 vtvt,...,1,360716,367216.0,great,2021-04-01T13:33:40.733,True,8397,54225,False,False



Informações gerais (types, não nulos):
<class 'pandas.core.frame.DataFrame'>
Index: 5316 entries, 0 to 7398
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   car_name             5316 non-null   object 
 1   yr_mfr               5316 non-null   int64  
 2   fuel_type            5316 non-null   object 
 3   kms_run              5316 non-null   int64  
 4   sale_price           5316 non-null   int64  
 5   city                 5316 non-null   object 
 6   times_viewed         5316 non-null   int64  
 7   body_type            5306 non-null   object 
 8   transmission         5042 non-null   object 
 9   variant              5316 non-null   object 
 10  assured_buy          5316 non-null   bool   
 11  registered_city      5314 non-null   object 
 12  registered_state     5314 non-null   object 
 13  is_hot               5316 non-null   bool   
 14  rto                  5316 non-null   object 
 15  sou

In [6]:
print("Estatísticas descritivas (variáveis numéricas):")
display(df.describe())

Estatísticas descritivas (variáveis numéricas):


,yr_mfr,kms_run,sale_price,times_viewed,total_owners,broker_quote,original_price,emi_starts_from,booking_down_pymnt
count,5316.000000,5316.000000,5.316000e+03,5316.000000,5316.000000,5.316000e+03,3.046000e+03,5316.000000,5316.000000
mean,2013.844996,61259.430211,4.142918e+05,1389.183597,1.310008,3.953369e+05,4.933909e+05,9622.965952,62143.920805
std,2.949186,42994.159086,1.921765e+05,1884.787168,0.556008,2.035752e+05,2.249658e+05,4463.775951,28826.490973
min,2000.000000,749.000000,0.000000e+00,3.000000,1.000000,0.000000e+00,1.262250e+05,0.000000,0.000000
25%,2012.000000,31397.000000,2.824990e+05,521.750000,1.000000,2.570000e+05,3.310572e+05,6562.000000,42375.000000
50%,2014.000000,53939.500000,3.702490e+05,989.000000,1.000000,3.500475e+05,4.405460e+05,8600.000000,55537.500000
75%,2016.000000,82343.750000,4.978240e+05,1720.000000,2.000000,4.835802e+05,5.979840e+05,11563.500000,74673.750000
max,2021.000000,996609.000000,1.533481e+06,61930.000000,5.000000,1.571598e+06,1.780955e+06,35619.000000,230023.000000


## 2. Distribuição da variável alvo: `sale_price`

Antes de modelar, precisamos entender a distribuição de `sale_price`:
- Faixa de valores
- Presença de outliers
- Se a distribuição é muito assimétrica (skewed), o que pode motivar transformações (ex.: log).

In [ ]:
print("Descrição descritiva da coluna 'sale_price':")
display(df["sale_price"].describe())

# Histograma com KDE
plt.figure(figsize=(12, 6))
sns.histplot(df["sale_price"], bins=50, kde=True)
plt.title("Distribuição de Sale Price")
plt.xlabel("Sale Price (Rúpias)")
plt.ylabel("Frequência")
plt.show()

# Boxplot para ver outliers
plt.figure(figsize=(10, 4))
sns.boxplot(x=df["sale_price"])
plt.title("Boxplot de Sale Price")
plt.xlabel("Sale Price (Rúpias)")
plt.show()

## 3. Análise de valores ausentes

Agora vamos verificar:
- Quantos valores ausentes existem em cada coluna
- Qual o percentual de ausência
- Quais colunas exigem uma estratégia de tratamento mais cuidadosa

In [ ]:
# Quantidade e percentual de valores ausentes por coluna
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    "Total Ausentes": missing_values,
    "Percentual (%)": missing_percent
})

print("Resumo de valores ausentes por coluna (apenas colunas com NaN):")
display(
    missing_df[missing_df["Total Ausentes"] > 0]
    .sort_values("Percentual (%)", ascending=False)
)
